In [11]:
import boto3
import logging
from botocore.exceptions import ClientError
from botocore.client import Config

In [12]:
s3 = boto3.resource('s3')

In [13]:
session = boto3.session.Session()

In [14]:
file_path = 'C:/Users/Administrator/Pictures/Camera Roll/love.jpg'
key_name = 'love.jpg'
buket_name = 'chensi'
access_key = 'CFDB3730D1E61E878E7D'
secret_key = 'WzhDMURDRDcxMTNBNjNBRTdBREQ2NzE2RTM2NzFEMTdCNDk5OURENkJd'
endpoint = 'http://chensi.scuts3.depts.bingosoft.net:29999/'


In [15]:
s3 = session.client('s3',use_ssl=False, endpoint_url=endpoint,
                   aws_access_key_id = access_key,
                   aws_secret_access_key=secret_key,
                   config=Config(signature_version='s3v4',s3={'addressing_style':'path'}))

In [20]:
#response = s3.create_bucket(Bucket='chensi01')


In [17]:
s3.upload_file(file_path,buket_name,key_name,ExtraArgs={'ContentType':'picture/jpg'})

In [20]:
from boto.s3.connection import S3Connection,OrdinaryCallingFormat,SubdomainCallingFormat
import ssl
import boto.s3.key
import os
import json
import random
import fire
import math
from filechunkio import FileChunkIO

In [23]:
class S3(object):
    def __init__(self,host,access_key_id,secret_access_key,port=443,is_secure=True):
        if is_secure:
            ssl._create_default_https_context =ssl._create_unverified_context
        self._secure=is_secure
          self._secure=is_secure
        self.host=host
        self.access_key_id=access_key_id
        self.secret_access_key=secret_access_key
        self.port=port
        self._connection=False
        self._client=None
    # def __del__(self):
    #    self._client.closed()
    def connect(self,aws=False):
        try:
            if aws:
                calling_format=SubdomainCallingFormat()
            else:
                calling_format=OrdinaryCallingFormat()
            conn = S3Connection(
                host=self.host,
                aws_access_key_id=self.access_key_id,
                aws_secret_access_key=self.secret_access_key,
                port=self.port,
                debug=1,
                is_secure=self._secure,
                calling_format=calling_format
            )
            self._connection=True
            self._client=conn
        except Exception as e:
            self._connection=False
            print(e)
    def upload_file(self,bucket,file_path,path=''):
        try:
            bucket=self._client.get_bucket(bucket)
            key=bucket.new_key(path+os.path.basename(file_path))
            key.set_contents_from_filename(file_path)
            return True
        except Exception as e:
            print(e)
            return False
    def chunk_upload_file(self,bucket,file_path):
        try:
            source_path = file_path
            source_size = os.stat(file_path).st_size
            chunk_size = 52428800
            chunk_count = int(math.ceil(source_size / float(chunk_size)))
            bucket=self._client.get_bucket(bucket)
            mp=bucket.initiate_multipart_upload(os.path.basename(file_path))
            for i in range(chunk_count):
                offset = chunk_size * i
                bytes_ = min(chunk_size, source_size - offset)
                with FileChunkIO(source_path, 'r', offset=offset,bytes=bytes_) as fp:
                    mp.upload_part_from_file(fp, part_num=i + 1)

            mp.complete_upload()
            return True
        except Exception as e:
            print(e)
            return False